In [1]:
import csv
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn import metrics
from mpl_toolkits import mplot3d
from numpy import set_printoptions
from numpy import absolute
from numpy import mean
from numpy import std



from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

#import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import utils
from tensorflow.keras.models import model_from_json

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
#from matplotlib import pyplot

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
    
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.axes3d import get_test_data
import seaborn as sns
%matplotlib inline

In [2]:
prosody_data = pd.read_csv('prosodic_features_M.csv ')
turker_score = pd.read_csv('Turker_Score_M.csv')

In [3]:
dA = pd.read_csv('prosodic_features_M2.csv ')
yRH = prosody_data['RecommendHiring']
yF = prosody_data['Friendly']
ySA = turker_score['StructuredAnswers']
yNS = prosody_data['NotStressed']
cRH = prosody_data['Hired']
cF = prosody_data['Friendly_c']
cSA = prosody_data['StructuredAnswers_c']
yNS = turker_score['NotStressed']

yCLG= turker_score['Colleague']
yEXTD = turker_score['Excited']
ySPKR = turker_score['SpeakingRate']
yNF = turker_score['NoFillers']
yPSD = turker_score['Paused']
yENGT =  turker_score['EngagingTone']
yCalm =  turker_score['Calm']
yNAWKD =turker_score['NotAwkward']
yFOC = turker_score['Focused']
yAUT = turker_score['Authentic']

MAUT = turker_score['Authentic_M']
MFOC = turker_score['Focused_M']
MNAWKD =turker_score['NotAwkward_M']
MNS = turker_score['NotStressed_M']
MCalm =  turker_score['Calm_M']
MENGT =  turker_score['EngagingTone_M']
MPSD = turker_score['Paused_M']
MNF =turker_score['NoFillers_M']
MSPKR = turker_score['SpeakingRate_M']
MEXTD = turker_score['Excited_M']
MRH = prosody_data['Hired_M']
MF= prosody_data['Friendly_M']
MENG = turker_score['Engaged_M']
MCLG =turker_score['Colleague_M'] 
MSA = turker_score['StructuredAnswers_M']

In [4]:
ranks = {}
from sklearn.preprocessing import MinMaxScaler
 
def rank_to_dict(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    #ranks = map(lambda x: round(x, 5), ranks)
    return dict(zip(names, ranks ))

In [5]:
def select_feature_kbest(prosody_data_y):
    array =  prosody_data.values

    X = prosody_data_X
    Y = prosody_data_y
    # feature extraction
    kbest_test = SelectKBest(score_func=f_classif, k=4)
    fit = kbest_test.fit(X, Y)
    # summarize scores
    set_printoptions(precision=3)
    #print(fit.scores_)
    features = fit.transform(X)
    # summarize selected features
    #print(features[0:5,:])

    ranks["kbest"] = rank_to_dict(np.abs(fit.scores_), prosody_data_X)
    rankDict = {}
    rankDict = ranks["kbest"]
    return rankDict

# Making arrays of Top 10 20 30 Features

In [6]:
def select_features(rankDict):
    selectedFeatures = []

    selectedFeatures = dict(sorted(rankDict.items(), key=lambda item: item[1],reverse=True))


    selectedFeatures_10 = []
    i=0
    for key in selectedFeatures.keys():
        if(i<10):
            selectedFeatures_10.append(key)
            i+=1

    selectedFeatures_20 = []
    i=0
    for key in selectedFeatures.keys():
        if(i<20):
            selectedFeatures_20.append(key)
            i+=1

    selectedFeatures_30 = []
    i=0
    for key in selectedFeatures.keys():
        if(i<30):
            selectedFeatures_30.append(key)
            i+=1
    return selectedFeatures_10,selectedFeatures_20,selectedFeatures_30


In [7]:
prosody_data_X = prosody_data.drop(['participant&question','Hired','RecommendHiring','Friendly','Friendly_M','Friendly_c','StructuredAnswers','StructuredAnswers_c','NotStressed','NotStressed_c','Colleague','Hired_M'],axis=1)

# Feature selection for using KBest

In [8]:
#feature ranks for 
prosody_data_y = yAUT
X_train, X_test, y_train, y_test = train_test_split(prosody_data_X, prosody_data_y, test_size=0.20, random_state=10)
selectedFeatures_Kbest=select_feature_kbest(yAUT)
print("\nScores of Features")
#selectedFeatures_RecommendHiring_Kbest


Scores of Features


In [9]:
selectedFeatures_kbest_10,selectedFeatures_kbest_20,selectedFeatures_kbest_30= select_features(selectedFeatures_Kbest)

#print("Top 10 Selected Features Using kbest:",selectedFeatures_kbest_10)

#print("\n\n\nTop 20 Selected Features Using kbest:",selectedFeatures_kbest_20)

#print("\n\n\nTop 30 Selected Features Using kbest:",selectedFeatures_kbest_30 )

# Applying Models

In [10]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)


In [11]:
def apply_NN_Moadel(no_of_inNeurons,no_of_hiddenLayerNeurons,no_of_epochs,selectedFeatures_No,d_Var):
    scaler = MinMaxScaler()
    X=prosody_data[selectedFeatures_No].values
    X =  scaler.fit_transform(X)
    Y=d_Var
    #encode class values as integers
    encoder = LabelEncoder()
    encoder.fit(Y)
    encoded_Y = encoder.transform(Y)
    #print(X)
    
    #print(encoded_Y)
    
    def baseline_model():
        # create model
        model = Sequential()
        model.add(Dense(no_of_hiddenLayerNeurons, input_dim=no_of_inNeurons, activation='relu'))
        model.add(Dense(no_of_hiddenLayerNeurons, input_dim=no_of_inNeurons, activation='relu'))
        model.add(Dense(no_of_hiddenLayerNeurons, input_dim=no_of_inNeurons, activation='relu'))
        model.add(Dense(no_of_hiddenLayerNeurons, input_dim=no_of_inNeurons, activation='relu'))
        model.add(Dense(no_of_hiddenLayerNeurons, input_dim=no_of_inNeurons, activation='relu'))
        model.add(Dense(3, activation='softmax'))
        # Compile model
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model
    
    estimator = KerasClassifier(build_fn=baseline_model, epochs=no_of_epochs, batch_size=5, verbose=0)
    kfold = KFold(n_splits=10, shuffle=True)
    results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
    print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

cvscores_NN_10_kbest= [] 
#(model_NN_10_kbest,cvscores_NN_10_kbest,FP_Rate_NN_10_kbest,TP_Rate_NN_10_kbest,auc_NN_10_kbest_) = 
apply_NN_Moadel(10,18,200,selectedFeatures_kbest_10,MF,TRH1,TRH2)


cvscores_NN_20_kbest= [] 
#(model_NN_10_kbest,cvscores_NN_10_kbest,FP_Rate_NN_10_kbest,TP_Rate_NN_10_kbest,auc_NN_10_kbest_) = 
apply_NN_Moadel(20,18,200,selectedFeatures_kbest_20,MF,TRH1,TRH2)

cvscores_NN_20_kbest= []  
#(model_NN_10_kbest,cvscores_NN_10_kbest,FP_Rate_NN_10_kbest,TP_Rate_NN_10_kbest,auc_NN_10_kbest_) = 
apply_NN_Moadel(20,18,200,selectedFeatures_kbest_20,MENG,TRH1,TRH2)

In [12]:
cvscores_NN_20_kbest= []  
#(model_NN_10_kbest,cvscores_NN_10_kbest,FP_Rate_NN_10_kbest,TP_Rate_NN_10_kbest,auc_NN_10_kbest_) = 
apply_NN_Moadel(20,18,200,selectedFeatures_kbest_20,MAUT)

Baseline: 71.76% (4.30%)


In [13]:
def Apply_Model(model,model_name,X_train,X_test,y_train,y_test):
    
    clf = model.fit(X_train,y_train)
    pred = model.predict(X_test)            
    
    # generate a no skill prediction (majority class)
    ns_probs = [0 for _ in range(len(y_test))]
    # fit a model
    model.fit(X_train,y_train)
    # predict probabilities
    m_probs = model.predict_proba(X_test)
        
    # keep probabilities for the positive outcome only
    m_probs = m_probs[:, 1]
    # calculate scores
    ns_auc = roc_auc_score(y_test, ns_probs)
    m_auc = roc_auc_score(y_test, m_probs)
    # summarize scores
    # print('No Skill: ROC AUC=%.3f' % (ns_auc))
    # print('Logistic: ROC AUC=%.3f' % (m_auc))
    # calculate roc curves
    ns_fpr, ns_tpr, _ = roc_curve(y_test, ns_probs)
    m_fpr, m_tpr, _ = roc_curve(y_test, m_probs)
    # plot the roc curve for the model
    plt.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
    plt.plot(m_fpr, m_tpr, marker='.', label=model_name)   
    # axis labels
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    # show the legend
    plt.legend()
    # show the plot
    plt.show()
    
    
    print(confusion_matrix(y_test,pred))
    print(classification_report(y_test,pred,zero_division=0))
    
    print('MSE',metrics.mean_squared_error(y_test,pred))
    
    if(model == svc_model):
      m_Cross = cross_val_score(model,X20,y20,cv=10)
    elif(model == knn):
      m_Cross = cross_val_score(model,X20,y20,cv=10)
    elif(model == rfc):
      m_Cross = cross_val_score(model,X20,y20,cv=10)
    print("Mean : ",m_Cross)
    print("CV Score  %.2f%% (+/- %.2f%%)" % (np.mean(m_Cross), np.std(m_Cross)))
    
    
    plt.figure(figsize = (10,6))
    sns.heatmap(confusion_matrix(y_test, pred), annot = True,fmt='d')
    plt.show()
    
    
    return (m_fpr, m_tpr,m_auc)

In [14]:
get_ipython().run_line_magic('matplotlib', 'inline')
X20=prosody_data[selectedFeatures_kbest_20]
y20=MRH
X20_train, X20_test, y20_train, y20_test = train_test_split(X20, y20, test_size=0.20, random_state=10)

In [15]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=600)
#(rfc_fpr, rfc_tpr, rfc_auc)=Apply_Model(rfc,"RFC",X20_train, X20_test, y20_train, y20_test)

cross_val_score(rfc, X20, y20,cv=10)


array([0.779, 0.809, 0.647, 0.838, 0.809, 0.75 , 0.809, 0.75 , 0.809,
       0.853])